# Случайный лес

--------
## Задача классификации

In [41]:
# Импортируем необходимые библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from collections import Counter

In [42]:
path = 'winequality-red.csv'
data_class = pd.read_csv(path)

print("Первые 5 строк датасета для классификации:")
print(data_class.head())
print(f"\nРазмер датасета: {data_class.shape}")

Первые 5 строк датасета для классификации:
   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  

### Создаем целевую переменную для классификации

In [43]:
data_class['wine_quality'] = data_class['quality'].apply(lambda x: 'good' if x >= 6 else 'bad')
data_class = data_class.drop('quality', axis=1)

print("\nРаспределение качества вина:")
print(data_class['wine_quality'].value_counts())


Распределение качества вина:
wine_quality
good    855
bad     744
Name: count, dtype: int64


### Разделим датасет на признаки и целевую переменную и масштабирование признаков

In [44]:
X = data_class.drop('wine_quality', axis=1)
y = data_class['wine_quality']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

print(f"Размер обучающей выборки: {X_train.shape}, Размер тестовой выборки: {X_test.shape}")

Размер обучающей выборки: (1279, 11), Размер тестовой выборки: (320, 11)


### Применим встроенный алгоритм для случайного леса 

In [45]:
rf_baseline = RandomForestClassifier(random_state=42)
rf_baseline.fit(X_train, y_train)

y_pred_baseline = rf_baseline.predict(X_test)

accuracy_baseline = accuracy_score(y_test, y_pred_baseline)
f1_baseline = f1_score(y_test, y_pred_baseline, average='weighted')

print("Бейзлайн Random Forest:")
print(f"Accuracy: {accuracy_baseline:.4f}")
print(f"F1-Score: {f1_baseline:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_baseline))

Бейзлайн Random Forest:
Accuracy: 0.8063
F1-Score: 0.8064

Classification Report:
              precision    recall  f1-score   support

         bad       0.78      0.81      0.80       149
        good       0.83      0.80      0.82       171

    accuracy                           0.81       320
   macro avg       0.81      0.81      0.81       320
weighted avg       0.81      0.81      0.81       320



### Подберем гиперпараметры для улучшения бейзлайна


##### `n_estimators: [50, 100, 200]`
- **Количество деревьев в ансамбле** - баланс точности и вычислительной стоимости
- **50**: быстрая тренировка, достаточная для начальной оценки
- **100**: стандартный выбор, хороший баланс производительности и точности
- **200**: большее разнообразие, потенциально лучшая обобщающая способность

##### `max_depth: [None, 10, 20, 30]`
- **Контроль глубины деревьев** - от полностью grown до ограниченных
- **None**: неограниченная глубина (полностью grown деревья)
- **10**: сильно ограниченные деревья, высокая bias, низкая variance
- **20**: умеренное ограничение, баланс сложности
- **30**: минимальное ограничение, близко к полным деревьям

##### `min_samples_split: [2, 5, 10]`
- **Минимальные samples для разделения** - контроль переобучения
- **2**: максимальная детализация (риск переобучения)
- **5**: умеренное ограничение, хороший компромисс
- **10**: сильное ограничение, устойчивость к шуму

##### `min_samples_leaf: [1, 2, 4]`
- **Минимальные samples в листьях** - стабилизация предсказаний
- **1**: максимальная гибкость (может ловить шум)
- **2**: стандартный выбор для баланса
- **4**: сглаженные предсказания, устойчивость к выбросам

##### Общая стратегия
Баланс между разнообразием ансамбля (n_estimators) и контролем сложности отдельных деревьев, что является ключевым для эффективности случайного леса.

In [46]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

best_params_rf = grid_search_rf.best_params_
print(f"Лучшие параметры: {best_params_rf}")

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Лучшие параметры: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


### Проведем обучение улучшенной модели и посчитаем метрики

In [47]:
best_rf = grid_search_rf.best_estimator_

y_pred_best_rf = best_rf.predict(X_test)

accuracy_best_rf = accuracy_score(y_test, y_pred_best_rf)
f1_best_rf = f1_score(y_test, y_pred_best_rf, average='weighted')

print("Улучшенный бейзлайн Random Forest:")
print(f"Accuracy: {accuracy_best_rf:.4f}")
print(f"F1-Score: {f1_best_rf:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_best_rf))

Улучшенный бейзлайн Random Forest:
Accuracy: 0.8125
F1-Score: 0.8126

Classification Report:
              precision    recall  f1-score   support

         bad       0.79      0.81      0.80       149
        good       0.83      0.82      0.82       171

    accuracy                           0.81       320
   macro avg       0.81      0.81      0.81       320
weighted avg       0.81      0.81      0.81       320



### Реализуем собственную имплементацию алгоритма

In [48]:
class CustomRandomForest:
    def __init__(self, n_estimators=100, max_depth=None, min_samples_split=2, 
                 min_samples_leaf=1, max_features='auto', random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.random_state = random_state
        self.trees = []
        self.feature_indices = [] 
        
    def _get_max_features(self, n_features):
        """Определяем количество признаков для каждого дерева"""
        if self.max_features == 'auto':
            return int(np.sqrt(n_features))
        elif self.max_features == 'sqrt':
            return int(np.sqrt(n_features))
        elif self.max_features == 'log2':
            return int(np.log2(n_features))
        elif isinstance(self.max_features, int):
            return min(self.max_features, n_features)
        elif isinstance(self.max_features, float):
            return int(self.max_features * n_features)
        else:
            return n_features
    
    def _bootstrap_sample(self, X, y, n_features):
        """Bootstrap sampling + случайный выбор признаков"""
        n_samples = X.shape[0]
        
        # Bootstrap sampling
        sample_indices = np.random.choice(n_samples, n_samples, replace=True)
        X_bootstrap = X[sample_indices]
        y_bootstrap = y[sample_indices]
        
        # Случайный выбор признаков
        feature_indices = np.random.choice(n_features, n_features, replace=False)
        selected_features = feature_indices[:self._get_max_features(n_features)]
        
        return X_bootstrap[:, selected_features], y_bootstrap, selected_features
    
    def fit(self, X, y):
        self.trees = []
        self.feature_indices = []
        X_array = np.array(X)
        y_array = np.array(y)
        n_samples, n_features = X_array.shape
        
        if self.random_state is not None:
            np.random.seed(self.random_state)
            
        for i in range(self.n_estimators):
            # Получаем bootstrap sample и случайные признаки
            X_sample, y_sample, selected_features = self._bootstrap_sample(X_array, y_array, n_features)
            
            tree = DecisionTreeClassifier(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf,
                random_state=self.random_state + i if self.random_state is not None else None
            )
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)
            self.feature_indices.append(selected_features)
            
    def predict(self, X):
        X_array = np.array(X)
        all_predictions = []
        
        for tree, features in zip(self.trees, self.feature_indices):
            # используем только те призанки, которые использовались при обучении дерева
            X_subset = X_array[:, features]
            predictions = tree.predict(X_subset)
            all_predictions.append(predictions)
        
        # Голосование большинством
        tree_predictions = np.array(all_predictions)
        final_predictions = []
        
        for sample_predictions in tree_predictions.T:
            most_common = Counter(sample_predictions).most_common(1)[0][0]
            final_predictions.append(most_common)
            
        return np.array(final_predictions)
    
    def predict_proba(self, X):
        X_array = np.array(X)
        all_probas = []
        
        for tree, features in zip(self.trees, self.feature_indices):
            X_subset = X_array[:, features]
            try:
                probas = tree.predict_proba(X_subset)
                all_probas.append(probas)
            except AttributeError:
                # Если дерево не поддерживает predict_proba, используем one-hot encoding
                predictions = tree.predict(X_subset)
                probas = np.zeros((len(predictions), len(tree.classes_)))
                for i, pred in enumerate(predictions):
                    class_idx = np.where(tree.classes_ == pred)[0][0]
                    probas[i, class_idx] = 1.0
                all_probas.append(probas)
        
        # Усредняем вероятности
        avg_probas = np.mean(all_probas, axis=0)
        return avg_probas

### Посмотрим метрики на дефолтных и улучшенных параметрах

In [49]:
custom_rf = CustomRandomForest(n_estimators=100, max_depth=None, random_state=42)
custom_rf.fit(X_train, y_train)

y_pred_custom_rf = custom_rf.predict(X_test)

accuracy_custom_rf = accuracy_score(y_test, y_pred_custom_rf)
f1_custom_rf = f1_score(y_test, y_pred_custom_rf, average='weighted')

print("Custom Random Forest (базовый):")
print(f"Accuracy: {accuracy_custom_rf:.4f}")
print(f"F1-Score: {f1_custom_rf:.4f}")

# Кастомный Random Forest с улучшенными параметрами
custom_rf_improved = CustomRandomForest(
    n_estimators=best_params_rf['n_estimators'],
    max_depth=best_params_rf['max_depth'],
    min_samples_split=best_params_rf['min_samples_split'],
    min_samples_leaf=best_params_rf['min_samples_leaf'],
    max_features='sqrt', 
    random_state=42
)
custom_rf_improved.fit(X_train, y_train)

y_pred_custom_rf_improved = custom_rf_improved.predict(X_test)

accuracy_custom_rf_improved = accuracy_score(y_test, y_pred_custom_rf_improved)
f1_custom_rf_improved = f1_score(y_test, y_pred_custom_rf_improved, average='weighted')

print("Custom Random Forest (улучшенный):")
print(f"Accuracy: {accuracy_custom_rf_improved:.4f}")
print(f"F1-Score: {f1_custom_rf_improved:.4f}")

Custom Random Forest (базовый):
Accuracy: 0.8000
F1-Score: 0.8002
Custom Random Forest (улучшенный):
Accuracy: 0.7969
F1-Score: 0.7970


#### Посмотрим на все результаты

In [50]:
print("\nСравнение результатов для задачи классификации:")
print(f"Бейзлайн Accuracy: {accuracy_baseline:.4f}, Улучшенный Accuracy: {accuracy_best_rf:.4f}, Custom RF Accuracy: {accuracy_custom_rf:.4f}, Custom RF улучшенный Accuracy: {accuracy_custom_rf_improved:.4f}")
print(f"Бейзлайн F1-Score: {f1_baseline:.4f}, Улучшенный F1-Score: {f1_best_rf:.4f}, Custom RF F1-Score: {f1_custom_rf:.4f}, Custom RF улучшенный F1-Score: {f1_custom_rf_improved:.4f}")


Сравнение результатов для задачи классификации:
Бейзлайн Accuracy: 0.8063, Улучшенный Accuracy: 0.8125, Custom RF Accuracy: 0.8000, Custom RF улучшенный Accuracy: 0.7969
Бейзлайн F1-Score: 0.8064, Улучшенный F1-Score: 0.8126, Custom RF F1-Score: 0.8002, Custom RF улучшенный F1-Score: 0.7970


------

## Задача регрессии

In [51]:
# Импортируем нужные библиотеки
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [52]:
path = 'hour.csv'
data_reg = pd.read_csv(path)

print("\nПервые 5 строк датасета для регрессии:")
print(data_reg.head())


Первые 5 строк датасета для регрессии:
   instant      dteday  season  yr  mnth  hr  holiday  weekday  workingday  \
0        1  2011-01-01       1   0     1   0        0        6           0   
1        2  2011-01-01       1   0     1   1        0        6           0   
2        3  2011-01-01       1   0     1   2        0        6           0   
3        4  2011-01-01       1   0     1   3        0        6           0   
4        5  2011-01-01       1   0     1   4        0        6           0   

   weathersit  temp   atemp   hum  windspeed  casual  registered  cnt  
0           1  0.24  0.2879  0.81        0.0       3          13   16  
1           1  0.22  0.2727  0.80        0.0       8          32   40  
2           1  0.22  0.2727  0.80        0.0       5          27   32  
3           1  0.24  0.2879  0.75        0.0       3          10   13  
4           1  0.24  0.2879  0.75        0.0       0           1    1  


### Разделяем на признаки и целевую переменную

In [53]:
# убрал дату в строковом представлении (есть дата в числовых колонках), так же число, которое предсказываем (cnt)
useful_columns = ['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 
                  'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed']

# Разделим данные на тестовую и обучающую выборку
X = data_reg[useful_columns]
y = data_reg['cnt']

print(f"\nИспользуемые признаки: {list(X.columns)}")
print(f"Размер X: {X.shape}")


Используемые признаки: ['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed']
Размер X: (17379, 12)


### Делим и масштабируем данные

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Размер обучающей выборки: {X_train.shape}, Размер тестовой выборки: {X_test.shape}")

scaler_reg = StandardScaler()
X_train_scaled = scaler_reg.fit_transform(X_train)
X_test_scaled = scaler_reg.transform(X_test)

Размер обучающей выборки: (13903, 12), Размер тестовой выборки: (3476, 12)


### Обучение бейзлайна и оценка модели

In [55]:
rf_baseline = RandomForestRegressor(random_state=42)
rf_baseline.fit(X_train_scaled, y_train)
y_pred_reg = rf_baseline.predict(X_test_scaled)

def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mae, mse, r2

print("\nБейзлайн Random Forest Regressor:")
mae_baseline, mse_baseline, r2_baseline = evaluate_model(y_test, y_pred_reg)
print(f"MAE: {mae_baseline:.4f}, MSE: {mse_baseline:.4f}, R^2: {r2_baseline:.4f}")


Бейзлайн Random Forest Regressor:
MAE: 24.8923, MSE: 1769.2754, R^2: 0.9441


### Подбираем гиперпараметры для улучшения

In [56]:
param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
}

grid_search = GridSearchCV(
    RandomForestRegressor(random_state=42),
    param_grid,
    cv=5,
    scoring="r2",
    verbose=1,
    n_jobs=-1,
)
grid_search.fit(X_train_scaled, y_train)
best_params_reg = grid_search.best_params_
print(f"Лучшие параметры: {best_params_reg}")

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Лучшие параметры: {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


### Обучим модели с подобранными гиперпараметрами

In [57]:
rf_optimized = grid_search.best_estimator_
rf_optimized.fit(X_train_scaled, y_train)
y_pred_optimized = rf_optimized.predict(X_test_scaled)

mae_optimized, mse_optimized, r2_optimized = evaluate_model(y_test, y_pred_optimized)

print("\nУлучшенный Random Forest Regressor:")
print(f"MAE: {mae_optimized:.4f}, MSE: {mse_optimized:.4f}, R²: {r2_optimized:.4f}")


Улучшенный Random Forest Regressor:
MAE: 24.7033, MSE: 1746.1439, R²: 0.9449


### Реализуем свой алгоритм

In [58]:
class CustomRandomForestRegressor:
    def __init__(self, n_estimators=100, max_depth=None, min_samples_split=2, min_samples_leaf=1, random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.random_state = random_state
        self.trees = []

    def _bootstrap_sample(self, X, y):
        X = np.array(X)
        y = np.array(y)

        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, n_samples, replace=True)
        return X[indices], y[indices]

    def fit(self, X, y):
        self.trees = []
        if self.random_state is not None:
            np.random.seed(self.random_state)
            
        for i in range(self.n_estimators):
            X_sample, y_sample = self._bootstrap_sample(X, y)
            tree = DecisionTreeRegressor(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf,
                random_state=self.random_state + i if self.random_state is not None else None
            )
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        return np.mean(predictions, axis=0)

### Посмотрим на результаты

In [59]:
custom_rf_reg = CustomRandomForestRegressor(n_estimators=100, max_depth=None, random_state=42)
custom_rf_reg.fit(X_train_scaled, y_train)
y_pred_custom_reg = custom_rf_reg.predict(X_test_scaled)

mae_custom, mse_custom, r2_custom = evaluate_model(y_test, y_pred_custom_reg)

print("\nCustom Random Forest Regressor (базовый):")
print(f"MAE: {mae_custom:.4f}, MSE: {mse_custom:.4f}, R²: {r2_custom:.4f}")

custom_rf_reg_improved = CustomRandomForestRegressor(
    n_estimators=best_params_reg['n_estimators'],
    max_depth=best_params_reg['max_depth'],
    min_samples_split=best_params_reg['min_samples_split'],
    min_samples_leaf=best_params_reg['min_samples_leaf'],
    random_state=42
)
custom_rf_reg_improved.fit(X_train_scaled, y_train)
y_pred_custom_reg_improved = custom_rf_reg_improved.predict(X_test_scaled)

mae_custom_improved, mse_custom_improved, r2_custom_improved = evaluate_model(y_test, y_pred_custom_reg_improved)

print("\nCustom Random Forest Regressor (улучшенный):")
print(f"MAE: {mae_custom_improved:.4f}, MSE: {mse_custom_improved:.4f}, R²: {r2_custom_improved:.4f}")


Custom Random Forest Regressor (базовый):
MAE: 24.9830, MSE: 1775.5030, R²: 0.9439

Custom Random Forest Regressor (улучшенный):
MAE: 24.8189, MSE: 1751.6846, R²: 0.9447


### Посмотрим на все результаты суммарно и сравним их

In [60]:
print("\nСравнение результатов для задачи регрессии:")
print(f"Бейзлайн MAE: {mae_baseline:.4f}, Улучшенный MAE: {mae_optimized:.4f}, Custom RF MAE: {mae_custom:.4f}, Custom RF улучшенный MAE: {mae_custom_improved:.4f}")
print(f"Бейзлайн MSE: {mse_baseline:.4f}, Улучшенный MSE: {mse_optimized:.4f}, Custom RF MSE: {mse_custom:.4f}, Custom RF улучшенный MSE: {mse_custom_improved:.4f}")
print(f"Бейзлайн R^2: {r2_baseline:.4f}, Улучшенный R^2: {r2_optimized:.4f}, Custom RF R^2: {r2_custom:.4f}, Custom RF улучшенный R^2: {r2_custom_improved:.4f}")


Сравнение результатов для задачи регрессии:
Бейзлайн MAE: 24.8923, Улучшенный MAE: 24.7033, Custom RF MAE: 24.9830, Custom RF улучшенный MAE: 24.8189
Бейзлайн MSE: 1769.2754, Улучшенный MSE: 1746.1439, Custom RF MSE: 1775.5030, Custom RF улучшенный MSE: 1751.6846
Бейзлайн R^2: 0.9441, Улучшенный R^2: 0.9449, Custom RF R^2: 0.9439, Custom RF улучшенный R^2: 0.9447
